In [37]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.metrics import roc_auc_score
import os

def build(diameter=4, nbits=2048):
    df = pd.read_csv('data/pcba.csv.gz', sep=',').set_index(['mol_id','smiles'])

    table = []
    for col in df.columns:
        negative, positive = df.groupby(col).size()
        table.append([col, df[col].notnull().sum(), negative, positive])
    table = pd.DataFrame(table, columns=['AID', 'count', 'negative', 'positive'])
    table['diff'] = np.abs(table['positive'] - table['negative'])
    table = table.sort_values(['diff', 'count'], ascending=True)
    display(table)
    
    col = table.iloc[0, 0]
    print(col)
    
    X, y = [], []
    for index, row in df.loc[df[col].notnull(), :].iterrows():
        print('\r%d/%d' % (len(y), df[col].notnull().sum()), end='')
        mol_id, smiles = index
        mol = Chem.MolFromSmiles(smiles)
        fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(
            mol, int(diameter/2), nBits=nbits, useChirality=False,
            useBondTypes=True, useFeatures=False)
        fp = np.asarray(fp)
        X.append(fp)
        y.append(row[col])
        
    X = np.asarray(X)
    y = np.asarray(y)
    
    return X, y

In [38]:
np.random.seed(123)

X, y = build()
print(X.shape, y.shape)

,AID,count,negative,positive,diff
111,PCBA-884,10471,7048,3423,3625
115,PCBA-899,8256,6419,1837,4582
114,PCBA-891,7888,6309,1579,4730
110,PCBA-883,8158,6921,1237,5684
121,PCBA-915,8051,7610,441,7169
...,...,...,...,...,...
70,PCBA-602310,394179,393869,310,393559
73,PCBA-624170,398628,397791,837,396954
107,PCBA-743266,399106,398800,306,398494
75,PCBA-624173,401002,400511,491,400020


PCBA-884
10470/10471(10471, 2048) (10471,)


In [50]:
#dnn.validation(aid, data, layers=[2000, 100],
#        batch_size=1000, nb_epoch=100, optimizer='Adam', lr=0.0001, activation='relu', 
#        dropout=0)

def validation(X_train, y_train, X_valid, y_valid):
    print(X.shape, y.shape)

    model = Sequential()

    model.add(Dense(1000, input_dim=X_train.shape[1], kernel_initializer='uniform'))
    model.add(Activation('sigmoid'))

    model.add(Dense(1000, kernel_initializer='uniform'))
    model.add(Activation('sigmoid'))

    model.add(Dense(1, kernel_initializer='uniform'))
    model.add(Activation('sigmoid'))

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mean_squared_error', optimizer=sgd)

    model.fit(X_train, y_train, epochs=100, batch_size=100)
    score = model.evaluate(X_valid, y_valid, batch_size=100)

    for i, layer in enumerate(model.layers):
        print(i, len(layer.get_weights()))

    auc = roc_auc_score(y_valid, model.predict_proba(X_valid))
    
    print('AUC\t%.3f\n' % (auc))

In [51]:
validation(X, y, X, y)

(10471, 2048) (10471,)
Epoch 1/100
10471/10471 [==============================] - 6s 549us/step - loss: 0.2215
Epoch 2/100
10471/10471 [==============================] - 6s 545us/step - loss: 0.2204
Epoch 3/100
10471/10471 [==============================] - 5s 499us/step - loss: 0.2194 0s - loss: 0.
Epoch 4/100
10471/10471 [==============================] - 6s 558us/step - loss: 0.2183
Epoch 5/100
10471/10471 [==============================] - 6s 566us/step - loss: 0.2167
Epoch 6/100
10471/10471 [==============================] - 6s 559us/step - loss: 0.2153
Epoch 7/100
10471/10471 [==============================] - 5s 514us/step - loss: 0.2125
Epoch 8/100
10471/10471 [==============================] - 5s 505us/step - loss: 0.2091
Epoch 9/100
10471/10471 [==============================] - 5s 434us/step - loss: 0.2049
Epoch 10/100
10471/10471 [==============================] - 5s 432us/step - loss: 0.2000
Epoch 11/100
10471/10471 [==============================] - 5s 442us/step - loss: 

10471/10471 [==============================] - 5s 485us/step - loss: 0.0731
Epoch 93/100
10471/10471 [==============================] - 5s 461us/step - loss: 0.0726
Epoch 94/100
10471/10471 [==============================] - 5s 439us/step - loss: 0.0726
Epoch 95/100
10471/10471 [==============================] - 5s 439us/step - loss: 0.0724
Epoch 96/100
10471/10471 [==============================] - 5s 445us/step - loss: 0.0719
Epoch 97/100
10471/10471 [==============================] - 5s 485us/step - loss: 0.0718
Epoch 98/100
10471/10471 [==============================] - 5s 485us/step - loss: 0.0713 0s - loss: 0
Epoch 99/100
10471/10471 [==============================] - 6s 553us/step - loss: 0.0708
Epoch 100/100
10471/10471 [==============================] - 3s 282us/step
0 2
1 0
2 2
3 0
4 2
5 0
10471/10471 [==============================] - 3s 295us/step
AUC	0.952



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation

cls = RandomForestClassifier(n_estimators=100)

scores = cross_validation.cross_val_score(
            cls, data, target, cv=4, scoring='roc_auc')

print(scores.mean())